### **Introducción a LangChain**

LangChain es una librería de alto nivel diseñada para facilitar la construcción de aplicaciones impulsadas por modelos de lenguaje (LLM), proporcionando abstracciones sobre prompts, cadenas (Chains), agentes y herramientas. 

Su objetivo es ofrecer componentes reutilizables que orquesten flujos de llamada a LLM, gestión de contexto, almacenamiento de estados conversacionales y encadenamiento de tareas complejas sin tener que lidiar con detalles de bajo nivel de la API.

In [ ]:
!pip install transformers langchain accelerate

#### **Agentes LangChain  y herramientas**

Los **Agents** en LangChain combinan LLMs con "herramientas" externas (por ejemplo, buscadores Web, bases de datos, funciones personalizadas). 

Funcionan así: el agente decide, a partir de un prompt y su memoria, qué herramienta invocar y con qué argumentos; tras ejecutar la herramienta, incorpora la respuesta al contexto y continúa el flujo. 

Esto permite construir asistentes que responden de manera dinámica, consultan APIs o ejecutan código.

El patrón general para una aplicación con LangChain incluye:

1. **PromptTemplate**: definiciones parametrizadas de prompts.
2. **LLMChain**: encadena el prompt con la llamada al modelo y procesa la respuesta.
3. **Memory** (opcional): guarda fragmentos de la conversación o metadatos.
4. **Agents & Tools** (opcional): permite reactividad a eventos o consultas externas.

**Creación de un resumidor de artículos de noticias**

Para un resumen de noticias con LangChain:

- `TextSplitter`: dividir artículos largos en segmentos manejables.
- `PromptTemplate`: crear plantillas como "Resume este fragmento en 2-3 oraciones."
- `LLMChain:` iterar sobre cada fragmento y agregar los subtítulos.
- `OutputParser:` combinar cada subtotal en un resumen coherente, filtrando duplicados y conectores.

#### **Ejemplo**

Invocamos el pipeline de generación de texto y, dado que distintas versiones de pipelines de Transformers pueden devolver formatos diferentes (una lista de cadenas, una lista de diccionarios, un único diccionario o incluso una cadena suelta), incluye una pequeña rutina de detección para extraer siempre el texto generado de forma segura.

In [ ]:
out = generate_text(
    "En este cuaderno, discutiremos los primeros pasos con IA generativa en Python.",
    max_new_tokens=50,
    do_sample=False
)

# Determina dónde está el texto generado
if isinstance(out, list):
    # Si es lista de strings
    if isinstance(out[0], str):
        texto_generado = out[0]
    # Si fuera lista de dicts (otras pipelines)
    elif isinstance(out[0], dict):
        texto_generado = out[0].get("generated_text") or out[0].get("generated_sequence")
    else:
        texto_generado = str(out[0])
else:
    # Si devuelve directamente un string o dict
    if isinstance(out, dict):
        texto_generado = out.get("generated_text") or out.get("generated_sequence") or str(out)
    else:
        texto_generado = str(out)

print(texto_generado)


**Ejemplo 1: Generación de una historia interactiva**

Este ejemplo genera una historia interactiva donde el lector puede elegir diferentes opciones y el modelo continuará la historia según la elección del lector.

In [ ]:
from transformers import pipeline
import torch

# 1. Detectae el dispositivo y tipo de dato óptimo
if torch.cuda.is_available():
    dispositivo = 0                     # usa GPU 0
    dtype = torch.bfloat16             # bfloat16 en GPU para velocidad y menor uso de memoria
else:
    dispositivo = -1                   # usa CPU
    dtype = torch.float32              # float32 en CPU para evitar conversiones lentas

# 2. Crea el pipeline de generación sólo una vez
generate_text = pipeline(
    "text-generation",                 
    model="aisquared/dlite-v1-355m",   
    torch_dtype=dtype,                 
    trust_remote_code=True,            
    device=dispositivo,                
)

# 3. Warm-up: primera llamada rápida para cache interno
_ = generate_text("Hola")

# 4. Función optimizada para continuar la historia
def continue_story(prompt: str, choice: str) -> str:
    """
    Genera la siguiente parte de la historia combinando el prompt y la elección del usuario.
    - max_new_tokens: límite bajo para tokens nuevos (reduce tiempo de generación)
    - do_sample=False: decodificación greedy (más rápida y determinista)
    """
    salida = generate_text(
        f"{prompt} {choice}",
        max_new_tokens=60,
        do_sample=False
    )
    # Algunos pipelines devuelven lista de strings
    if isinstance(salida, list) and isinstance(salida[0], str):
        return salida[0]
    # Otros devuelven lista de dicts con key "generated_text" o "generated_sequence"
    if isinstance(salida, list) and isinstance(salida[0], dict):
        return salida[0].get("generated_text") or salida[0].get("generated_sequence", "")
    # Fallback: cast a str
    return str(salida)

# 5. Flujo interactivo con comentarios en español
story_prompt = (
    "Te encuentras en un bosque oscuro. Delante de ti hay dos caminos. "
    "¿Tomas el camino izquierdo o el camino derecho?"
)
print(story_prompt)

choice1 = input("Ingresa tu elección (izquierdo/derecho): ").strip().lower()
story1 = continue_story(story_prompt, choice1)
print(story1)

story_prompt2 = (
    story1 +
    "\nTe topas con una figura misteriosa. ¿Te acercas a la figura o te escondes detrás de un árbol?"
)
choice2 = input("Ingresa tu elección (acercarse/esconderse): ").strip().lower()
story2 = continue_story(story1, choice2)
print(story2)

story_prompt3 = (
    story2 +
    "\nLa figura te ofrece un objeto mágico. ¿Lo aceptas o lo rechazas?"
)
choice3 = input("Ingresa tu elección (aceptar/rechazar): ").strip().lower()
story3 = continue_story(story2, choice3)
print(story3)

story_prompt4 = (
    story3 +
    "\nCon el objeto mágico en tu mano, sientes un poder. ¿Lo usas ahora o lo guardas para más tarde?"
)
choice4 = input("Ingresa tu elección (usar/guardar): ").strip().lower()
story4 = continue_story(story3, choice4)
print(story4)

story_prompt5 = (
    story4 +
    "\nTe encuentras con un dragón feroz. ¿Luchas contra él o intentas comunicarte?"
)
choice5 = input("Ingresa tu elección (luchar/comunicar): ").strip().lower()
story5 = continue_story(story4, choice5)
print(story5)

print("\nLa historia termina aquí. ¡Gracias por jugar!")


**Ejemplo 2: Generación de un documento técnico**

Este ejemplo utiliza el modelo para generar un documento técnico sobre inteligencia artificial, dividiendo el contenido en secciones específicas.

In [ ]:
from transformers import pipeline
import torch

# 1. Detecta el dispositivo y tipo de dato óptimo
if torch.cuda.is_available():
    dispositivo = 0                     # usa GPU 0
    dtype = torch.bfloat16             # bfloat16 en GPU para velocidad y menor uso de memoria
else:
    dispositivo = -1                   # usa CPU
    dtype = torch.float32              # float32 en CPU para evitar conversiones lentas

# 2. Crea el pipeline de generación de texto una sola vez
generate_text = pipeline(
    "text-generation",
    model="aisquared/dlite-v1-355m",
    torch_dtype=dtype,
    trust_remote_code=True,
    device=dispositivo,
)

# 3. Warm-up: llama brevemente para inicializar caches internas
_ = generate_text("Hola")

def _extract_text(output) -> str:
    """
    Función interna para extraer el texto generado
    Soporta lista de strings, lista de dicts o un string directo.
    """
    if isinstance(output, list):
        first = output[0]
        if isinstance(first, str):
            return first
        if isinstance(first, dict):
            return first.get("generated_text") or first.get("generated_sequence", "")
    if isinstance(output, dict):
        return output.get("generated_text") or output.get("generated_sequence", "")
    return str(output)

# 4. Función optimizada para generar secciones del documento
def generate_section(title: str) -> str:
    """
    Genera una sección detallada sobre el título dado,
    usando un prompt contextualizado en IA.
    """
    prompt = f"Escriba una sección detallada sobre {title} en el contexto de inteligencia artificial."
    out = generate_text(
        prompt,
        max_new_tokens=100,   # límite de tokens a generar por sección
        do_sample=False        # decodificación greedy (más rápida y determinista)
    )
    return _extract_text(out)

# 5. Generación del documento con comentarios en español
title = "Una guía completa sobre la inteligencia artificial"
print(title)
print("=" * len(title))

for section_title in [
    "Introduction",
    "History of Artificial Intelligence",
    "Machine Learning",
    "Deep Learning",
    "Applications of AI",
    "Future of AI"
]:
    print(f"\n{section_title}")
    print("-" * len(section_title))
    texto = generate_section(section_title)
    print(texto)

# 6. Sección de conclusión
print("\nConclusion")
print("-" * len("Conclusion"))
out_conclusion = generate_text(
    "Escriba una conclusión para una guía completa sobre inteligencia artificial.",
    max_new_tokens=80,
    do_sample=False
)
print(_extract_text(out_conclusion))


**Ejemplo 3: Explicaciones y ejemplos de funciones matemáticas**

Este ejemplo proporcionará una explicación de varios conceptos matemáticos y ejemplos prácticos para cada uno.

In [ ]:
from transformers import pipeline
import torch

# 1. Detecta dispositivo y tipo de dato óptimo
if torch.cuda.is_available():
    dispositivo = 0                   # usa GPU 0
    dtype = torch.bfloat16           # bfloat16 en GPU para velocidad y menor uso de memoria
else:
    dispositivo = -1                 # usa CPU
    dtype = torch.float32            # float32 en CPU para evitar conversiones lentas

# 2. Crea el pipeline de generación de texto una sola vez
generate_text = pipeline(
    "text-generation",
    model="aisquared/dlite-v1-355m",
    torch_dtype=dtype,
    trust_remote_code=True,
    device=dispositivo,
)

# 3. Warm-up: inicializar caches internas
_ = generate_text("Hola")

def _extract_text(output) -> str:
    """
    Extrae el texto generado de distintos formatos de salida:
    - lista de strings
    - lista de dicts con 'generated_text' o 'generated_sequence'
    - dict único
    - string directo
    """
    if isinstance(output, list):
        first = output[0]
        if isinstance(first, str):
            return first
        if isinstance(first, dict):
            return first.get("generated_text") or first.get("generated_sequence", "")
    if isinstance(output, dict):
        return output.get("generated_text") or output.get("generated_sequence", "")
    return str(output)

# 4. Función optimizada para explicar conceptos matemáticos
def explain_math_concept(concept: str) -> str:
    """
    Genera una explicación detallada del concepto matemático con ejemplos.
    """
    prompt = f"Explica el {concept} con ejemplos."
    out = generate_text(
        prompt,
        max_new_tokens=100,    # límite de tokens para la explicación
        do_sample=False        # decodificación greedy (más rápida)
    )
    return _extract_text(out)

# 5. Función optimizada para generar ejemplos prácticos
def generate_math_examples(concept: str) -> str:
    """
    Proporciona ejemplos prácticos y soluciones paso a paso para el concepto dado.
    """
    prompt = f"Proporciona ejemplo y soluciones paso a paso para el {concept}."
    out = generate_text(
        prompt,
        max_new_tokens=150,    # límite de tokens para los ejemplos
        do_sample=False
    )
    return _extract_text(out)

# 6. Lista de conceptos y generación
math_concepts = ["derivatives", "integrals", "linear algebra", "probability", "statistics"]

for concept in math_concepts:
    print(f"\nConcepto: {concept.capitalize()}")
    print("=" * (len(concept) + 10))

    # Explicación del concepto
    explanation = explain_math_concept(concept)
    print("\nExplicación:")
    print(explanation)

    # Ejemplos prácticos
    examples = generate_math_examples(concept)
    print("\nEjemplos prácticos:")
    print(examples)

    print("\n" + "=" * 80)

# 7. Ejemplo adicional: ecuación cuadrática
quadratic_prompt = (
    "Resuelve la ecuación cuadrática 3x^2 - 4x - 5 = 0."
    "Proporciona una solución y una explicación paso a paso."
)
quadratic_out = generate_text(
    quadratic_prompt,
    max_new_tokens=120,
    do_sample=False
)
print("\nEjemplo de ecuación cuadrática:")
print("-" * 30)
print(_extract_text(quadratic_out))


#### **Ejercicios**

1. Diseña una historia interactiva con al menos tres niveles de decisiones. Escribe un diagrama de flujo que muestre todas las posibles rutas y resultados de la historia.
2. Lee varias historias generadas por el modelo basadas en diferentes elecciones. Evalúa la continuidad y coherencia de las historias. Identifica y discute cualquier inconsistencia que encuentres.

3. Escribe manualmente al menos dos nuevas escenas para cada posible elección en el segundo nivel de decisiones de la historia interactiva proporcionada. Asegúrate de que las escenas sean coherentes y se integren bien en la narrativa existente.

4. Diseña un esquema detallado para un documento técnico sobre un tema de tu elección en inteligencia artificial. Incluye al menos seis secciones principales y describe brevemente el contenido de cada sección.

5. Utiliza el modelo para generar una sección sobre un tema técnico de tu elección. Analiza el contenido generado en términos de precisión, relevancia y profundidad. ¿Cuáles son las fortalezas y debilidades del contenido generado?
6. Encuentra un artículo técnico real sobre un tema similar al generado por el modelo. Compara ambos textos en términos de calidad, detalle y claridad. ¿Qué mejoras sugieres para el contenido generado por el modelo?

7. Elige un concepto matemático avanzado (como transformadas de Fourier o series de Taylor). Escribe una explicación detallada del concepto, utilizando al menos dos ejemplos prácticos y resolviendo un problema paso a paso.
8. Toma una solución generada por el modelo para un problema matemático (por ejemplo, la solución de una ecuación cuadrática). Revisa y valida cada paso de la solución. ¿Es correcta? Si encuentras errores, corrígelos y explica el proceso correcto.
9. Escribe un conjunto de problemas matemáticos que se puedan resolver utilizando los conceptos explicados por el modelo. Para cada problema, proporciona una solución detallada y discute cómo el modelo podría ayudar a generar soluciones similares.
10. Investiga diferentes modelos de lenguaje (por ejemplo, GPT-3, BERT, T5). Compara sus arquitecturas, capacidades y aplicaciones. ¿Cuáles son las principales diferencias y en qué contextos se
11. Discute las implicaciones éticas y sociales del uso de modelos de lenguaje para generar contenido. Considera aspectos como la generación de noticias falsas, la privacidad de los datos y el sesgo en los modelos. ¿Qué medidas se pueden tomar para mitigar estos riesgos?
12. Diseña un proyecto que utilice un modelo de lenguaje para resolver un problema específico en una industria (por ejemplo, salud, educación, finanzas). Describe el problema, cómo el modelo ayudará a resolverlo y los pasos necesarios para implementar la solución.



In [ ]:
## Tus respuestas

### **Prompting con LangChain**

#### **1. Prompt Templates: separación de redacción y variables**

Las **Prompt Templates** son plantillas parametrizables que distinguen claramente la **estructura fija** de un prompt (instrucción, formatos, estilo) del **contenido dinámico** (variables concretas).

* **Desacoplamiento**
  El texto base del prompt queda centralizado en una plantilla; las variables se inyectan al ejecutarlo. De este modo no mezclamos la lógica de generación con valores específicos, lo que facilita su lectura y modificación.

* **Mantenibilidad**
  Al cambiar el tono, corregir errores tipográficos o ajustar el estilo, basta con actualizar la plantilla; todas las invocaciones posteriores heredarán la mejora de forma automática.

* **Reutilización**
  Una misma plantilla puede servir para diferenciar tareas (resúmenes, explicaciones, preguntas) simplemente variando las variables de entrada. Esto reduce la duplicación de prompts en el código y promueve un diseño más limpio.


#### **2. Few-Shot & ExampleSelectors: ejemplos manuales y automáticos**

Los **Few-Shot Prompts** incluyen ejemplos concretos de pares *(entrada -> salida)* dentro del propio prompt, enseñando al modelo "en vuelo" el patrón deseado. Sin embargo, elegir manualmente qué ejemplos incluir puede resultar laborioso y no siempre óptimo.

* **ExampleSelectors**
  Automatizan la selección de los ejemplos más relevantes desde una librería amplia, basándose en:

  * **Similitud semántica** (mide la cercanía con embeddings).
  * **Coincidencia de palabras clave** o metadatos.

* **Beneficios**

  1. **Adaptabilidad**: el modelo recibe ejemplos alineados con la consulta actual, mejorando su entendimiento.
  2. **Escalabilidad**: permite gestionar grandes bancos de ejemplos sin saturar el contexto.
  3. **Calidad constante**: evita sesgos o errores de selección manual, pues el sistema elige según métricas objetivas.


#### **3. Output Parsers: formatos rígidos y fiabilidad**

Los **Output Parsers** convierten la respuesta "libre" de un LLM en estructuras de datos controladas (JSON, diccionarios tipados, objetos de validación).

* **Validación temprana**
  Si la salida no encaja en el esquema esperado (por ejemplo, falta un campo obligatorio o el tipo de dato es incorrecto), el parser detecta la anomalía y permite:

  * Solicitar al modelo un reintento con instrucciones aclaratorias.
  * Aplicar reglas de recuperación o fallback.

* **Reducción de errores downstream**
  Al garantizar que el formato de salida sea siempre uniforme, evitamos excepciones e inconsistencias en etapas posteriores (almacenamiento, visualización, integración con otras APIs).

* **Trazabilidad y observabilidad**
  Registrar cuántas veces fallan los parsers o qué validaciones son las más conflictivas ayuda a iterar y optimizar tanto los prompts como los esquemas de datos.


#### **4. Aplicación en un resumidor de noticias**

Al incorporar estas tres piezas en un **News Articles Summarizer**, elevamos la calidad y consistencia de los resúmenes:

1. **Prompt Templates**

   * Definen el estilo editorial (longitud, tono, nivel de detalle) en un único lugar.
   * Permiten ajustar rápidamente el "brief" que se envía al LLM sin tocar la lógica de orquestación.

2. **Few-Shot & ExampleSelectors**

   * Recurre a resúmenes humanos previos de artículos similares para guiar el estilo y evitar alucinaciones.
   * Selecciona los ejemplos más alineados semánticamente con el tema del artículo, garantizando coherencia temática.

3. **Output Parsers**

   * Fuerzan un formato JSON fijo:

     ```json
     {
       "title": "...",
       "summary": "...",
       "keywords": ["...", "..."],
       "author": "...",
       "date": "YYYY-MM-DD"
     }
     ```
   * Facilitan la integración con sistemas de publicación o bases de datos, minimizando la limpieza de datos manual.


#### **5. Extensión a Knowledge Graphs**

Para convertir los resúmenes en un **recurso navegable y semántico**, podemos construir grafos de conocimiento:

1. **Extracción de entidades y relaciones**

   * Mediante prompts especializados, pedimos al LLM que identifique nombres de personas, organizaciones, conceptos clave y sus interacciones.

2. **Construcción del grafo**

   * **Nodos**: las entidades detectadas.
   * **Aristas**: relaciones etiquetadas (e.g., "colaboró con", "es parte de", "causó").

3. **Consultas semánticas**

   * Combinamos un *retriever* para localizar fragmentos relevantes en los resúmenes con prompts que interpretan rutas en el grafo:

     > "¿Cómo se relaciona el concepto X con la organización Y?"

4. **Beneficios**

   * Descubrimos conexiones ocultas entre temas.
   * Ofrecemos a los usuarios exploración interactiva: navegar de nodo en nodo, profundizar en relaciones, generar nuevos insights.

